In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

In [3]:
df = pd.read_csv('/content/creditcard.csv')

In [4]:
df = (df - df.min()) / (df.max() - df.min())

In [5]:
X = df.drop(columns=['Class'])
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42
)

clf = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'
)
clf.fit(X_train, y_train)

importances = clf.feature_importances_
feat_imp = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
})
feat_imp = feat_imp.sort_values('importance', ascending=False).reset_index(drop=True)

print(feat_imp)

# plt.figure(figsize=(8, 10))
# plt.barh(feat_imp['feature'], feat_imp['importance'])
# plt.gca().invert_yaxis()
# plt.title('Feature Importances')
# plt.xlabel('Importance')
# plt.tight_layout()
# plt.show()

   feature  importance
0      V14    0.185077
1       V4    0.120695
2      V10    0.116924
3      V12    0.101880
4      V17    0.089537
5       V3    0.064627
6      V11    0.046515
7      V16    0.042677
8       V2    0.036178
9       V9    0.026259
10     V21    0.022648
11      V7    0.013823
12     V18    0.011518
13     V19    0.011364
14  Amount    0.009923
15      V6    0.008869
16      V5    0.008617
17     V27    0.008098
18     V13    0.007603
19     V28    0.007559
20     V26    0.007484
21     V22    0.006798
22      V8    0.006705
23     V15    0.006655
24      V1    0.006035
25     V20    0.005969
26     V25    0.005746
27     V23    0.005162
28    Time    0.004622
29     V24    0.004432


In [6]:
features = feat_imp['feature'][:6].to_list()
features_with_class =  features + ['Class']
print(features)
print(features_with_class)

['V14', 'V4', 'V10', 'V12', 'V17', 'V3']
['V14', 'V4', 'V10', 'V12', 'V17', 'V3', 'Class']


In [7]:
df['Class'].unique()

array([0., 1.])

In [8]:
df['Class'].value_counts()

,count
Class,
0.0,284315
1.0,492


In [9]:
df = df[features_with_class].copy(deep=True)

In [10]:
df.describe()

,V14,V4,V10,V12,V17,V3,Class
count,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000
mean,0.646053,0.251930,0.508722,0.704193,0.731130,0.837414,0.001727
std,0.032231,0.062764,0.022528,0.037660,0.024678,0.026275,0.041527
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.631744,0.214311,0.497644,0.688907,0.717074,0.821985,0.000000
50%,0.647755,0.251050,0.506800,0.709471,0.729221,0.840530,0.000000
75%,0.662635,0.284882,0.518113,0.727494,0.742743,0.855213,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import pytorch_lightning as pl

from torch import nn
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, train_test_split

In [ ]:
!pip install lightning

In [106]:
device = 'cuda'
# wtf ?
batch_size = 1024
precision_model = '64'

data_precision_torch = torch.float16
match precision_model:
    case 'bf16-mixed':
        data_precision_torch = torch.float16
    case '16':
        data_precision_torch = torch.float16
    case '32-mixed':
        data_precision_torch = torch.float32
    case '32':
        data_precision_torch = torch.float32
    case '64':
        data_precision_torch = torch.float64

num_workers = 7
epochs = 150
# name_batch_precision_epochs
model_name = f'titanic_{str(batch_size)}_{str(precision_model)}_{str(epochs)}.pt'

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V14     284807 non-null  float64
 1   V4      284807 non-null  float64
 2   V10     284807 non-null  float64
 3   V12     284807 non-null  float64
 4   V17     284807 non-null  float64
 5   V3      284807 non-null  float64
 6   Class   284807 non-null  float64
dtypes: float64(7)
memory usage: 15.2 MB


In [16]:
df['Class'].unique()

array([0., 1.])

In [107]:
class IsFraud(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(6, 128)
        self.layer_2 = nn.Linear(128, 256)
        self.layer_3 = nn.Linear(256, 512)
        self.layer_4 = nn.Linear(512, 1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.leaky_relu(self.layer_1(x))
        x = F.leaky_relu(self.layer_2(x))
        x = F.leaky_relu(self.layer_3(x))
        return self.layer_4(x).squeeze(1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds.float() == y).float().mean()
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [67]:
class IsFraudDataset(Dataset):
    def __init__(self, data, targets, data_precision=torch.float64):
        self.data = torch.tensor(data.values, dtype=data_precision)
        self.targets = torch.tensor(targets.values, dtype=data_precision)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [19]:
class IsFraudDataModule(pl.LightningDataModule):
    def __init__(self,
                 dataset,
                 batch_size=64,
                 num_workers=8,
                 n_splits=5,
                 fold_idx=0,
                 shuffle_train=True,
                 data_precision=torch.float64):
        super().__init__()
        self.dataset = dataset
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.n_splits = n_splits
        self.fold_idx = fold_idx
        self.shuffle_train = shuffle_train
        self.data_precision = data_precision

    def setup(self, stage=None):
        # setup train data
        df = self.dataset

        # setup target
        X = df.drop(columns=["Class"])
        y = df["Class"]

        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            test_size=0.3,
                                                            stratify=y,
                                                            random_state=42)

        # FIXME crossvalidation K-Fold ?
        kf = KFold(n_splits=self.n_splits, shuffle=True, random_state=42)
        splits = list(kf.split(X_train))
        train_idx, val_idx = splits[self.fold_idx]
        self.train_dataset = IsFraudDataset(X_train.iloc[train_idx],
                                            y_train.iloc[train_idx],
                                            data_precision=self.data_precision)
        self.val_dataset = IsFraudDataset(X_train.iloc[val_idx],
                                          y_train.iloc[val_idx],
                                          data_precision=self.data_precision)

        # setup test data
        self.test_dataset = torch.tensor(X_test.values, dtype=self.data_precision)

    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          persistent_workers=True,
                          shuffle=self.shuffle_train)

    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          persistent_workers=True,
                          num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset,
                          batch_size=self.batch_size,
                          persistent_workers=True,
                          num_workers=self.num_workers)

In [65]:
df['Class'].unique()

array([0., 1.])

In [108]:
model = IsFraud()
logger = TensorBoardLogger("tb_logs", name="my_model")

checkpoint_callback = ModelCheckpoint(
    # ?
    monitor="val_acc",
    dirpath="checkpoints/",
    filename="best-checkpoint",
    save_top_k=3,
    mode="min"
)

dm = IsFraudDataModule(
    dataset=df,
    batch_size=batch_size,
    num_workers=num_workers,
    data_precision=data_precision_torch
)

trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback],
    max_epochs=epochs,
    accelerator='gpu' if torch.cuda.is_available() and device == 'cuda' else 'cpu',
    precision=precision_model,
    detect_anomaly=False,
    accumulate_grad_batches=batch_size
)
trainer.fit(model, datamodule=dm)

torch.save(model.state_dict(), model_name)
print(f"Model saved to {model_name}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type   | Params | Mode 
-------------------------------------------
0 | layer_1 | Linear | 896    | train
1 | layer_2 | Linear | 33.0 K | train
2 | layer_3 | Linear | 131 K  | train
3 | layer_4 | Linear | 513    | train
-------------------------------------------
166 K     Trainable params
0         Non-trainable params
166 K     Total params
0.664     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eva

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 7 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=150` reached.


Model saved to titanic_1024_64_150.pt


In [109]:
model = IsFraud()
model.load_state_dict(torch.load(model_name))
model.to(device)
model.eval()

IsFraud(
  (layer_1): Linear(in_features=6, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=512, bias=True)
  (layer_4): Linear(in_features=512, out_features=1, bias=True)
)

In [110]:
# make metrics
y_true = []
y_pred = []
idx = 0

with torch.no_grad():
    for idx in range(len(df)):
        # print(df.iloc[idx])
        x = df.iloc[idx][:6].values  # extract features as NumPy array
        x = torch.tensor(x, dtype=torch.float32).unsqueeze(0).to(device)  # shape (1, 6)
        # print(x)
        outputs = model(x)
        # print(df.iloc[idx][6], outputs.cpu().numpy()[0])
        # break
        # print(x, outputs)
        y_true.append(df['Class'][idx])
        y_pred.append(outputs.cpu().numpy()[0])

1.0 3.6325378e-09
1.0 3.8127816e-09
1.0 3.23039e-09
1.0 4.4784554e-09
1.0 3.8137418e-09
1.0 4.0909547e-09
1.0 4.250465e-09
1.0 4.9067586e-09
1.0 5.1702678e-09
1.0 4.200497e-09
1.0 4.428205e-09
1.0 5.4816898e-09
1.0 4.110885e-09
1.0 3.2642322e-09
1.0 3.1109642e-09
1.0 5.0476574e-09
1.0 3.4172083e-09
1.0 5.1024456e-09
1.0 3.5179168e-09
1.0 4.5091406e-09
1.0 2.966748e-09
1.0 3.1408884e-09
1.0 3.9033856e-09
1.0 3.5767185e-09
1.0 5.656506e-09
1.0 4.0921253e-09
1.0 3.7547445e-09
1.0 4.096264e-09
1.0 2.9136475e-09
1.0 3.3209469e-09
1.0 4.0075e-09
1.0 3.3691157e-09
1.0 3.7259937e-09
1.0 3.7259937e-09
1.0 3.7217889e-09
1.0 3.7217889e-09
1.0 4.1754706e-09
1.0 2.8537903e-09
1.0 2.929255e-09
1.0 3.1324408e-09
1.0 5.5074807e-09
1.0 2.7648015e-09
1.0 3.22421e-09
1.0 3.9509422e-09
1.0 5.0715423e-09
1.0 3.3346124e-09
1.0 5.1136104e-09
1.0 3.5335852e-09
1.0 5.141121e-09
1.0 3.157328e-09
1.0 3.277133e-09
1.0 3.745067e-09
1.0 3.2286342e-09
1.0 3.6164507e-09
1.0 4.6244084e-09
1.0 3.0444915e-09
1.0 3.72132

In [121]:
idx = 0
for x in y_true:
  if x > 0:
    print(f"{x}, {y_pred[idx]:.10f}")
    idx = idx + 1

1.0, 0.0000000036
1.0, 0.0000000038
1.0, 0.0000000032
1.0, 0.0000000045
1.0, 0.0000000038
1.0, 0.0000000041
1.0, 0.0000000043
1.0, 0.0000000049
1.0, 0.0000000052
1.0, 0.0000000042
1.0, 0.0000000044
1.0, 0.0000000055
1.0, 0.0000000041
1.0, 0.0000000033
1.0, 0.0000000031
1.0, 0.0000000050
1.0, 0.0000000034
1.0, 0.0000000051
1.0, 0.0000000035
1.0, 0.0000000045
1.0, 0.0000000030
1.0, 0.0000000031
1.0, 0.0000000039
1.0, 0.0000000036
1.0, 0.0000000057
1.0, 0.0000000041
1.0, 0.0000000038
1.0, 0.0000000041
1.0, 0.0000000029
1.0, 0.0000000033
1.0, 0.0000000040
1.0, 0.0000000034
1.0, 0.0000000037
1.0, 0.0000000037
1.0, 0.0000000037
1.0, 0.0000000037
1.0, 0.0000000042
1.0, 0.0000000029
1.0, 0.0000000029
1.0, 0.0000000031
1.0, 0.0000000055
1.0, 0.0000000028
1.0, 0.0000000032
1.0, 0.0000000040
1.0, 0.0000000051
1.0, 0.0000000033
1.0, 0.0000000051
1.0, 0.0000000035
1.0, 0.0000000051
1.0, 0.0000000032
1.0, 0.0000000033
1.0, 0.0000000037
1.0, 0.0000000032
1.0, 0.0000000036
1.0, 0.0000000046
1.0, 0.000

In [105]:
threshold_down = -6.75
threshold_up = -9.3
y_pred_cl = []
for x in y_pred:
  if x > threshold_up and x < threshold_down:
    x = 0
  else:
    x = 1
  y_pred_cl.append(x)

print("Accuracy:", accuracy_score(y_true, y_pred_cl))
print("\nClassification Report:\n", classification_report(y_true, y_pred_cl))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred_cl))

Accuracy: 0.9091209134606945

Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.91      0.95    284315
         1.0       0.02      0.92      0.03       492

    accuracy                           0.91    284807
   macro avg       0.51      0.92      0.49    284807
weighted avg       1.00      0.91      0.95    284807


Confusion Matrix:
 [[258470  25845]
 [    38    454]]
